## Regularizer

심층 신경망은 자유도가 굉장히 높습니다.

따라서 이전까지, 수많은 하이퍼파라미터를 튜닝(keras tuner)하고, 시각화(tensorboard)하는 방법까지 알아보았습니다.

또한 우리는 규제 방법 2가지를 이미 알고있습니다.

1) Early_Stop : 최고의 규제방법입니다.

2) Batch Normalization : 일부 규제의 효과를 지니고 있습니다

In [309]:
import keras
import tensorflow as tf
from zipfile import ZipFile
from pandas import read_csv
from pathlib import Path
import io
with ZipFile("../Data/고객 대출등급 분류 해커톤.zip","r") as zpf:
    data=io.BytesIO(zpf.read("고객 대출등급 분류 해커톤/train.csv"))

data=read_csv(data)

### L1, L2 규제

l1,l2 규제는 신경망의 연결 가중치를 조절합니다.

1) l1 :  많은 가중치를 0으로 만든 희소모델을 만듬

2) l2 :  회소성을 만들지 않는 방향으로 조절

> 앞서 보았지만 Adam 과 Adam의 변형은 l2 규제와 같이 사용하지 않습니다. 가중치 감쇠의 효과를 보고싶다면 AdamW 를 사용했습니다

In [310]:
regularize=keras.regularizers.l1(0.01)

In [311]:
from functools import partial

l1_layer=partial(keras.layers.Dense,activation="relu",kernel_initializer="he_normal",kernel_regularizer=regularize)
normal_layer=partial(keras.layers.Dense,activation="relu",kernel_initializer="he_normal")

In [312]:
X=data[["대출금액","총상환이자"]]
y=data[["연간소득"]]

In [313]:
l1_model=keras.Sequential([
    l1_layer(units=2),l1_layer(units=2),l1_layer(units=1)
])

normal_model=keras.Sequential(
    [normal_layer(units=2),normal_layer(units=2),normal_layer(units=1)]
)

In [314]:
mse=keras.losses.MeanSquaredError()
metrics=[keras.metrics.MeanSquaredError()]
opt=keras.optimizers.SGD(0.01,momentum=0.9,nesterov=True)

In [315]:
l1_model.compile(opt,loss=mse,metrics=metrics)
normal_model.compile(opt,loss=mse,metrics=metrics)

In [316]:
loss=l1_model(X)

In [317]:
l1_model.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.021048436>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.036124215>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.019073315>]

In [318]:
loss2=normal_model(X)

In [319]:
normal_model.losses

[]

위처럼 기본적으로 규제값을 기본적으로 loss 로 가지고있음을 알수있습니다

### Dropout

심층 신경망에서 가장 인기있는 규제방법입니다.

매 훈련 스텝에서 $p$확률로 뉴런을 드롭아웃시킵니다(출력뉴런을 제외한 입력뉴런도 포함시킵니다) 

이는 각 뉴런이 더 유연한 값을 가지게끔 유도합니다.

입력층 입장에서 한번 살펴보죠

In [320]:
dropout=keras.layers.Dropout(rate=0.75) # 75%확률로 출력값을 0으로 바꿉니다

In [321]:
X[1:4]

,대출금액,총상환이자
1,14400000,234060.0
2,12000000,151944.0
3,14400000,153108.0


In [322]:
dropout(X,training=True)[1:4]

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[57600000.,        0.],
       [48000000.,        0.],
       [       0.,   612432.]], dtype=float32)>

In [323]:
dropout(X,training=False)[1:4]

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[14400000.,   234060.],
       [12000000.,   151944.],
       [14400000.,   153108.]], dtype=float32)>

위의 값을 보면 dropout으로 나온 입력값이 다름을 알 수 있습니다.

이는 훈련이 끝난 이후 다음층의 뉴런을 생각하면 이해할 수 있습니다.

훈련을 진행한다고 가정하면, 위의 상황에서 $input=[input1,input2]$ 는 다음과 같은 기댓값을 가집니다.($p$ : 드롭아웃 확률)

$\bar {input_i}= (1-p)*input_i$ ($i=1,2$)

즉 각 입력값의 기댓값이 원래의 입력값과 같게 조정해줄 필요가 있습니다. 따라서 입력값에 $\frac{1}{1-p}$를 곱해 이를 조정해줍니다

드롭아웃을 사용할 떄 유의할 점은 다음과 같습니다.

1) 모든 은닉층에 드롭아웃을 적용하는게 지나치게 강한 규제라 판단되면, 마지막 은닉층 뒤에만 더합니다

2) Selu를 사용할 경우, 드롭아웃을 사용하고싶다면, "alpha-dropout"(드롭아웃의 변형)을 사용해야 자기정규성을 유지할 수 있습니다

### MC Dropout

몬테 카를로 드롭아웃은 앙상블과 비슷한 과정을 가집니다.

기본적으로 dropout 층은 훈련이 끝난 후 `training=False`로 전환됩니다. 이를 `training=True`로 변환한 뒤 

`T`번의 추론결과를 사용하는 것이 MC dropout의 원리입니다

In [324]:
import numpy as np
X=data.select_dtypes(np.number)

In [325]:
X.head(3)

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,12480000,72000000,18.90,15,0,0,0.0,0.0,0.0
1,14400000,130800000,22.33,21,0,373572,234060.0,0.0,0.0
2,12000000,96000000,8.60,14,0,928644,151944.0,0.0,0.0


In [326]:
y=data[["대출등급"]]

In [327]:
y.nunique()

대출등급    7
dtype: int64

In [328]:
stringlookup=keras.layers.StringLookup(max_tokens=7,num_oov_indices=0)
stringlookup.adapt(y)

In [329]:
proceed_y=stringlookup(y).numpy()

In [330]:
base_layer_dump=lambda units : [keras.layers.Dropout(rate=0.4)
                                ,keras.layers.Dense(units,activation="relu",kernel_initializer="he_normal")]

In [331]:
norm_layer=keras.layers.Normalization()
norm_layer.adapt(X.to_numpy())

layers_dump=[[keras.Input(shape=(9,)),norm_layer]]
for _ in range(2):
    layers_dump.append(base_layer_dump(16))

layers_dump.append([keras.layers.Dropout(rate=0.4),
                    keras.layers.Dense(units=7,activation="softmax",kernel_initializer="glorot_normal")])

In [332]:
simple_model=keras.Sequential(sum(layers_dump,[]))

In [333]:
simple_model.summary()

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_12                │ (None, 9)              │            19 │
│ (Normalization)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_57 (Dropout)            │ (None, 9)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_93 (Dense)                │ (None, 16)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_58 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_94 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_59 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_95 (Dense)                │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 570 (2.23 KB)

 Trainable params: 551 (2.15 KB)

 Non-trainable params: 19 (80.00 B)

In [334]:
simple_model.compile(
    optimizer=keras.optimizers.AdamW(),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

In [336]:
history=simple_model.fit(X,proceed_y,batch_size=16,epochs=5,callbacks=[keras.callbacks.EarlyStopping(patience=5)],validation_split=0.2)

Epoch 1/5
4815/4815 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - loss: 2.2126 - sparse_categorical_accuracy: 0.2642 - val_loss: 1.4741 - val_sparse_categorical_accuracy: 0.3733
Epoch 2/5
4815/4815 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - loss: 1.5333 - sparse_categorical_accuracy: 0.3409 - val_loss: 1.4530 - val_sparse_categorical_accuracy: 0.3811
Epoch 3/5
4815/4815 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - loss: 1.5166 - sparse_categorical_accuracy: 0.3469 - val_loss: 1.4536 - val_sparse_categorical_accuracy: 0.3926
Epoch 4/5
4815/4815 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - loss: 1.5199 - sparse_categorical_accuracy: 0.3453 - val_loss: 1.4505 - val_sparse_categorical_accuracy: 0.3918
Epoch 5/5
4815/4815 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - loss: 1.5175 - sparse_categorical_accuracy: 0.3491 - val_loss: 1.4393 - val_sparse_categorical_accuracy: 0.3933


일반적인 predict 방법

In [ ]:
tf.argmax(simple_model.predict(X[:100]),axis=-1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 3, 0, 1, 0, 0, 0, 3, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 3, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0])>

MC dropout

문제는 model 안에 `BatchNormalization` 과 같이 학습할때마다 값을 조정하는 층이 있다면, 함부로 `model(X,training=True)`를 사용하면 안된다는 점입니다.

In [365]:
T=100

probas=np.stack([simple_model(X[9:10],training=True).numpy() for _ in range(T)])

In [368]:
probas.shape

(100, 1, 7)

아래가 MC dropout의 결과입니다

In [369]:
probas.mean(axis=0)

array([[0.3113089 , 0.28820422, 0.18989275, 0.1269572 , 0.06355434,
        0.01611806, 0.00396465]], dtype=float32)

In [371]:
probas.std(axis=0)

array([[0.03576138, 0.02560103, 0.0503832 , 0.02942962, 0.01917847,
        0.00678704, 0.00149401]], dtype=float32)

만일 batchnormalization 층이 포함되었다면 아래의 서브클래싱을 dropout층 대신 쓰입니다

In [374]:
class MCdropout(keras.layers.Dropout):
    def __init__(self, rate, noise_shape=None, seed=None, **kwargs):
        super().__init__(rate, noise_shape, seed, **kwargs)
    def call(self,inputs,training=False):
        return super().call(inputs,training=True)